In [2]:
import pandas as pd

# Charger le fichier CSV
file_path = '../data/population.csv'
df = pd.read_csv(file_path)
print(df.columns)

# Fonction simple pour convertir les coordonnées DMS vers décimal
def dms_to_decimal(coord_str):
    # Nettoyer la chaîne
    coord_str = coord_str.replace('"', '').strip()
    
    # Détecter la direction (E, O pour longitude, sinon c'est latitude Nord)
    direction = 'N'  # Par défaut
    if coord_str.endswith(' E') or coord_str.endswith(' O'):
        direction = coord_str[-1]  # Prendre le dernier caractère (E ou O)
        coord_str = coord_str[:-2]  # Enlever " E" ou " O"
    elif 'E' in coord_str:
        direction = 'E'
        coord_str = coord_str.replace('E', '')
    elif 'O' in coord_str:
        direction = 'O' 
        coord_str = coord_str.replace('O', '')
    
    # Remplacer les caractères spéciaux par des espaces
    coord_str = coord_str.replace('°', ' ').replace("'", ' ').replace('"', '')
    
    # Diviser en parties
    parts = [p.strip() for p in coord_str.split() if p.strip()]
    
    # Extraire degrés, minutes, secondes
    degrees = float(parts[0]) if len(parts) > 0 else 0
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    
    # Convertir en décimal
    decimal = degrees + minutes/60 + seconds/3600
    
    # Appliquer le signe pour Ouest
    if direction == 'O':
        decimal = -decimal
        
    return decimal

# Convertir LONGITUDE et LATITUDE en décimal
df['longitude_decimal'] = df['LONGITUDE'].apply(dms_to_decimal)
df['latitude_decimal'] = df['LATITUDE'].apply(dms_to_decimal)

# Créer la colonne geopoint au format "latitude,longitude"
df['geopoint'] = df['latitude_decimal'].astype(str) + ',' + df['longitude_decimal'].astype(str)

# Afficher quelques exemples
print("\nExemples de conversion :")
print(df[['Département', 'LONGITUDE', 'LATITUDE', 'geopoint']].head())

# Sauvegarder le DataFrame avec la nouvelle colonne dans un nouveau fichier CSV
output_file_path = '../data/population_with_geopoint.csv'
df.to_csv(output_file_path, index=False)

print(f"\nFichier sauvegardé : {output_file_path}")
print(f"Nouvelle colonne 'geopoint' ajoutée avec {len(df)} lignes")

Index(['IdDepartement', 'Département', '0 à 19 ans', '20 à 39 ans',
       '40 à 59 ans', '60 à 74 ans', '75 ans et plus', 'Total', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')

Exemples de conversion :
               Département   LONGITUDE   LATITUDE  \
0                      Ain  5°20'56" E  46°05'58"   
1                    Aisne  3°33'30" E  49°33'34"   
2                   Allier  3°11'18" E  46°23'37"   
3  Alpes-de-Haute-Provence  6°14'38" E  44°06'22"   
4             Hautes-Alpes  6°15'47" E  44°39'49"   

                               geopoint  
0  46.099444444444444,5.348888888888888  
1  49.559444444444445,3.558333333333333  
2   46.39361111111111,3.188333333333333  
3   44.10611111111111,6.243888888888889  
4   44.66361111111111,6.263055555555556  

Fichier sauvegardé : ../data/population_with_geopoint.csv
Nouvelle colonne 'geopoint' ajoutée avec 96 lignes
